In [1]:
import os

# for data aggregation.
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance, lonlat

# for data visualisation.
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.plotly as py
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')
import deckgljupyter.Layer as deckgl
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
access_token = 'pk.eyJ1IjoidG9wc29sMTciLCJhIjoiY2szOGQ3MWhoMDdtdDNqb3ZlNHBla2w4bSJ9.lgVDxlf8ndzEDe496xKi0Q'
view_options = {
    'center': [126.908, 37.132],
    'zoom': 12,
    'bearing': 0,
    'pitch': 60,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}

def get_random_color(opacity=1):
    """
        deckgl.Layer 파라미터 중 getColor 에 값을 주는데 쓰입니다.
    """
    rgba = np.random.choice(range(256), size=3).tolist()
    rgba.append(opacity*255)
    return rgba


def draw_path(path_list, name_list=None, colors=[], opacity=1): 
    """
        path_list 로 넘어온 path 들을 한 지도 안에 그립니다.
    """
    # 시각화할 데이터 형태로 가공
    data = []
    for i, path in enumerate(path_list):
        d = {
            'path': path,
            'color': get_random_color() if len(colors) == 0 else colors[i]
        }
        if name_list != None:
            d['name'] = name_list[i]
        data.append(d)
            
    # 지도 설정
    v = view_options.copy()
    v['center'] = np.mean(path_list[0], axis=0)
    
    # 지도그리기
    m = deckgl.Map(**view_options)
    m.add(deckgl.PathLayer(data,
                           getWidth=20,
                           getColor='obj => obj.color',
                           opacity=opacity,
                           pickable=True,
                           tooltip=['name']))
    
    m.show()

In [3]:
df_route_stations = pd.read_csv('PJT001_routestationinfo.csv')
df_route_stations.shape
df_route_stations.head()

(37831, 7)

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
0,65286,228000018,10-4,1,용인터미널,228001552,47634.0
1,65287,228000018,10-4,2,용인터미널(경유),277102443,NaN
2,65288,228000018,10-4,3,포브스병원,228000443,29439.0
3,65289,228000018,10-4,4,제일교회,228000665,29881.0
4,65290,228000018,10-4,5,라이프아파트,228000664,29457.0


In [4]:
stations = pd.read_csv('PJT001_stations_table.csv')

In [5]:
stations.head()

,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID
0,228003422,용인시,손골마을회관.국제학교,"Songol Community Center,",56443.0,노변정류장,경기도 용인시,NaN,37.342517,127.066817,56443.0,NaN
1,228003423,용인시,풀잎사랑,Pulipsarang,56444.0,노변정류장,경기도 용인시,NaN,37.341800,127.068983,56444.0,NaN
2,228003424,용인시,풀잎사랑,Pulipsarang,56445.0,노변정류장,경기도 용인시,NaN,37.341817,127.069083,56445.0,NaN
3,228003425,용인시,대성공정,Daesung Process,56446.0,노변정류장,경기도 용인시,NaN,37.339350,127.073067,56446.0,NaN
4,228003426,용인시,대성공정,Daesung Process,56447.0,노변정류장,경기도 용인시,NaN,37.339183,127.073400,56447.0,NaN


In [6]:
stations = stations[['표준정류장ID', '시군명', 'WGS84위도', 'WGS84경도']]

In [7]:
stations.columns = ['station_id', 'govern', 'lat', 'lng'] 

In [8]:
df_route_stations= pd.merge(df_route_stations, stations, how='left', on='station_id')

In [9]:
df_route_stations

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng
0,65286,228000018,10-4,1,용인터미널,228001552,47634.0,용인시,37.232783,127.210100
1,65286,228000018,10-4,1,용인터미널,228001552,47634.0,용인시,37.232783,127.210100
2,65287,228000018,10-4,2,용인터미널(경유),277102443,NaN,용인시,37.235350,127.209033
3,65288,228000018,10-4,3,포브스병원,228000443,29439.0,용인시,37.235267,127.210617
4,65289,228000018,10-4,4,제일교회,228000665,29881.0,용인시,37.234300,127.213333
...,...,...,...,...,...,...,...,...,...,...
39452,191750,241491011,38-1,63,신일해피트리후문,233002733,NaN,화성시,37.222650,126.978983
39453,191751,241491011,38-1,64,기배동행정복지센터,233002726,NaN,화성시,37.224000,126.983150
39454,191752,241491011,38-1,65,기안1통입구,233002959,NaN,화성시,37.225683,126.986250
39455,191753,241491011,38-1,66,배양2리,233002717,55455.0,화성시,37.226833,126.992067


In [10]:
line_ID = pd.read_csv('PJT001_routestationmapping.csv')

In [11]:
line_ID

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
0,경기시내,경원여객M,2805000,216000044,28050900,M6410
1,경기시내,강화운수,4100100,232000028,41001001,2
2,경기시내,강화운수,4100100,232000029,41001013,88
3,경기시내,강화운수,4100100,232000061,41001020,3000
4,경기시내,강화운수,4100100,232000067,41001024,388
...,...,...,...,...,...,...
2122,경기시내,서현운수,4108800,229000060,41088004,330
2123,경기시내,서현운수,4108800,229000063,41088005,850
2124,경기시내,코레일네트웍스,4108900,213000024,41089001,8507
2125,경기시내,신성교통,4109100,229000102,41091900,M7111


In [12]:
line_ID = line_ID[['이비노선ID', '표준노선ID']]

In [13]:
df_route_stations = pd.merge(df_route_stations, line_ID, how='left', left_on='pr_station_id', right_on='이비노선ID')

In [14]:
df_route_stations = df_route_stations[df_route_stations['govern'] == '화성시']

In [15]:
# ods[route_id] 에 route_id 노선의 전 구간을 set 에 담습니다.
ods = {}
grp = df_route_stations.groupby('표준노선ID')

for route_id, gdf in grp:
    gdf = gdf.sort_values('bus_line_no_seq')
    od_set = set()
    for i in range(1, len(gdf)):
        od = (gdf['station_id'].iloc[i-1], gdf['station_id'].iloc[i])
        od_set.add(od)
    ods[route_id] = od_set

In [16]:
# diff_routes[route_id1][route_id2] 에 노선 route_id1와 route_id2 간의 중복도 값을 구합니다.
diff_routes = {}
for route_id1, od_set1 in ods.items():
    d = {}
    
    for route_id2, od_set2 in ods.items():
        diff = od_set1 & od_set2
        d[route_id2] = len(diff) / len(od_set1)
    diff_routes[route_id1] = d

In [17]:
# 위에서 구한 두 노선의 중복도 값을 데이터프레임으로 만들어줍니다.
# 0~1의 값을 0~100(%) 로 바꿔줍니다.
df_redundancy = pd.DataFrame(diff_routes)
df_redundancy = df_redundancy * 100
df_redundancy.head()

,41002090.0,41003015.0,41006281.0,41008001.0,41008101.0,41008102.0,41008103.0,41008105.0,41008106.0,41008107.0,...,41086004.0,41086005.0,41086008.0,41086009.0,41086010.0,41086011.0,41086012.0,41086013.0,41086014.0,41086015.0
41002090.0,100.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,2.941176,22.727273,21.739130,0.0,12.962963,30.0,20.833333,7.142857,0.000000
41003015.0,0.0,100.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,2.941176,6.818182,4.347826,0.0,5.555556,10.0,6.250000,7.142857,15.789474
41006281.0,0.0,0.0,100.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,2.083333,0.000000,0.000000
41008001.0,0.0,0.0,0.0,100.000000,5.405405,0.0,2.857143,4.166667,2.777778,4.545455,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
41008101.0,0.0,0.0,0.0,2.173913,100.000000,0.0,42.857143,25.000000,83.333333,77.272727,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [18]:
# plt.figure(figsize=(18, 18))
# sns.heatmap(df_redundancy, square=True, annot=True, fmt='.0f', cmap='Blues', cbar=False, vmin=0, vmax=100)
# plt.show()

In [19]:
redundancy_flat = df_redundancy.values.flatten()
redundancy_flat = [i for i in redundancy_flat if i != 100]

In [20]:
pd.Series(redundancy_flat).iplot(kind='hist')

In [21]:
pd.Series(redundancy_flat).describe()

count    5848.000000
mean        5.057060
std        13.168150
min         0.000000
25%         0.000000
50%         0.000000
75%         2.857143
max        97.674419
dtype: float64

In [22]:
topn_pair = {}
for route_id1, others in diff_routes.items():
    for route_id2, redundancy in others.items():
        if route_id1 == route_id2:
            continue
        if redundancy >= 0.8 and (topn_pair.get((route_id2, route_id1), None) == None):
            topn_pair[(route_id1, route_id2)] = redundancy

In [23]:
len(topn_pair)

31

In [24]:
# 먼저 위에서 구한 노선쌍의 중복도를 데이터프레임으로 만듭니다.
df_topn_pair = pd.Series(topn_pair).to_frame().reset_index()
df_topn_pair.columns = ['route_1', 'route_2', 'redundancy']
df_topn_pair.sort_values('redundancy', ascending=False, inplace=True)
df_topn_pair.head()

,route_1,route_2,redundancy
30,41086001.0,41086015.0,1.000000
1,41008103.0,41008122.0,1.000000
7,41008124.0,41008126.0,0.976744
3,41008105.0,41008117.0,0.958333
4,41008107.0,41008106.0,0.954545


In [25]:
df_route_stations = df_route_stations.drop_duplicates()

In [26]:
df_route_stations.head()

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng,이비노선ID,표준노선ID
998,68834,228000204,24,53,한원IC,233000169,37403.0,화성시,37.152733,127.122883,228000204.0,41006281.0
999,68835,228000204,24,54,장지동,233000168,36808.0,화성시,37.155117,127.117900,228000204.0,41006281.0
1000,68836,228000204,24,55,장지동,233000167,36809.0,화성시,37.155867,127.115017,228000204.0,41006281.0
1001,68837,228000204,24,56,롯데캐슬.제일풍경채,233003130,55526.0,화성시,37.157483,127.109283,228000204.0,41006281.0
1002,68838,228000204,24,57,자이파밀리에,233003131,55527.0,화성시,37.159167,127.106050,228000204.0,41006281.0


In [27]:
TripChain = pd.read_csv('PJT001_TripChain.csv')

In [28]:
# 컬럼명 변경
TripChain.columns = ['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [29]:
TripChain['버스노선ID1']

0          41002045.0
1          41031040.0
2          41031121.0
3          41031013.0
4          41020001.0
              ...    
1048570    41067012.0
1048571    41061025.0
1048572    41036131.0
1048573    41036113.0
1048574    41036113.0
Name: 버스노선ID1, Length: 1048575, dtype: float64

In [30]:
df_topn_pair

,route_1,route_2,redundancy
30,41086001.0,41086015.0,1.000000
1,41008103.0,41008122.0,1.000000
7,41008124.0,41008126.0,0.976744
3,41008105.0,41008117.0,0.958333
4,41008107.0,41008106.0,0.954545
25,41008157.0,41008103.0,0.944444
27,41008157.0,41008145.0,0.944444
26,41008157.0,41008122.0,0.944444
6,41008122.0,41008145.0,0.942857
2,41008103.0,41008145.0,0.942857


In [45]:
count_route1 = []
for i in df_topn_pair['route_1']:
    
    count = 0
    
    if TripChain[TripChain['버스노선ID1']==i]['버스노선ID1'].count() > 0:
        count += TripChain[TripChain['버스노선ID1']==i]['버스노선ID1'].count()
        
    if TripChain[TripChain['버스노선ID2']==i]['버스노선ID2'].count() > 0:
        count += TripChain[TripChain['버스노선ID2']==i]['버스노선ID2'].count()
        
    if TripChain[TripChain['버스노선ID3']==i]['버스노선ID3'].count() > 0:
        count += TripChain[TripChain['버스노선ID3']==i]['버스노선ID3'].count()
        
    if TripChain[TripChain['버스노선ID4']==i]['버스노선ID4'].count() > 0:
        count += TripChain[TripChain['버스노선ID4']==i]['버스노선ID4'].count()
        
    if TripChain[TripChain['버스노선ID5']==i]['버스노선ID5'].count() > 0:
        count += TripChain[TripChain['버스노선ID5']==i]['버스노선ID5'].count()
        
    count_route1.append(count)

In [46]:
print(len(count_route1))
print(len(df_topn_pair))

31
31


In [47]:
df_topn_pair['count_route1'] = count_route1

In [48]:
count_route2 = []
for i in df_topn_pair['route_2']:
    count = 0
    if TripChain[TripChain['버스노선ID1']==i]['버스노선ID1'].count() > 0:
        count += TripChain[TripChain['버스노선ID1']==i]['버스노선ID1'].count()
    if TripChain[TripChain['버스노선ID2']==i]['버스노선ID2'].count() > 0:
        count += TripChain[TripChain['버스노선ID2']==i]['버스노선ID2'].count()
    if TripChain[TripChain['버스노선ID3']==i]['버스노선ID3'].count() > 0:
        count += TripChain[TripChain['버스노선ID3']==i]['버스노선ID3'].count()
    if TripChain[TripChain['버스노선ID4']==i]['버스노선ID4'].count() > 0:
        count += TripChain[TripChain['버스노선ID4']==i]['버스노선ID4'].count()
    if TripChain[TripChain['버스노선ID5']==i]['버스노선ID5'].count() > 0:
        count += TripChain[TripChain['버스노선ID5']==i]['버스노선ID5'].count()
    count_route2.append(count)

In [49]:
df_topn_pair['count_route2'] = count_route2

In [50]:
df_topn_pair

,route_1,route_2,redundancy,count_route1,count_route2
30,41086001.0,41086015.0,1.000000,237,558
1,41008103.0,41008122.0,1.000000,8,0
7,41008124.0,41008126.0,0.976744,24,4
3,41008105.0,41008117.0,0.958333,3,6
4,41008107.0,41008106.0,0.954545,0,0
25,41008157.0,41008103.0,0.944444,0,8
27,41008157.0,41008145.0,0.944444,0,1
26,41008157.0,41008122.0,0.944444,0,0
6,41008122.0,41008145.0,0.942857,0,1
2,41008103.0,41008145.0,0.942857,8,1


In [51]:
tmp = df_route_stations[['bus_line_no', '표준노선ID', '이비노선ID']]

In [55]:
tmp.columns = ['bus_line_no(1)', 'std_id', 'EB_id']

In [56]:
pairs = pd.merge(df_topn_pair, tmp, how='left', left_on = 'route_1', right_on = 'std_id')

In [57]:
pairs = pairs.drop_duplicates()

In [58]:
pairs

,route_1,route_2,redundancy,count_route1,count_route2,bus_line_no(1),std_id,EB_id
0,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0
35,41008103.0,41008122.0,1.000000,8,0,4,41008103.0,233000015.0
70,41008124.0,41008126.0,0.976744,24,4,6,41008124.0,233000040.0
114,41008105.0,41008117.0,0.958333,3,6,9,41008105.0,233000025.0
138,41008107.0,41008106.0,0.954545,0,0,2-4,41008107.0,233000029.0
160,41008157.0,41008103.0,0.944444,0,8,27,41008157.0,233000096.0
178,41008157.0,41008145.0,0.944444,0,1,27,41008157.0,233000096.0
196,41008157.0,41008122.0,0.944444,0,0,27,41008157.0,233000096.0
214,41008122.0,41008145.0,0.942857,0,1,4-1,41008122.0,233000038.0
249,41008103.0,41008145.0,0.942857,8,1,4,41008103.0,233000015.0


In [59]:
tmp.columns = ['bus_line_no(2)', 'std_id', 'EB_id']

In [61]:
pairs= pd.merge(pairs, tmp, how='left', left_on = 'route_2', right_on = 'std_id')

In [62]:
pairs

,route_1,route_2,redundancy,count_route1,count_route2,bus_line_no(1),std_id_x,EB_id_x,bus_line_no(2),std_id_y,EB_id_y
0,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0,6003,41086015.0,233000266.0
1,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0,6003,41086015.0,233000266.0
2,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0,6003,41086015.0,233000266.0
3,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0,6003,41086015.0,233000266.0
4,41086001.0,41086015.0,1.000000,237,558,6001,41086001.0,233000131.0,6003,41086015.0,233000266.0
...,...,...,...,...,...,...,...,...,...,...,...
1378,41008101.0,41008106.0,0.810811,5,0,2,41008101.0,233000013.0,2-3,41008106.0,233000030.0
1379,41008101.0,41008106.0,0.810811,5,0,2,41008101.0,233000013.0,2-3,41008106.0,233000030.0
1380,41008101.0,41008106.0,0.810811,5,0,2,41008101.0,233000013.0,2-3,41008106.0,233000030.0
1381,41008101.0,41008106.0,0.810811,5,0,2,41008101.0,233000013.0,2-3,41008106.0,233000030.0


In [65]:
pairs = pairs.drop(['std_id_x', 'EB_id_x', 'std_id_y', 'EB_id_y'], axis = 1)

In [67]:
pairs = pairs.drop_duplicates()

In [68]:
pairs

,route_1,route_2,redundancy,count_route1,count_route2,bus_line_no(1),bus_line_no(2)
0,41086001.0,41086015.0,1.000000,237,558,6001,6003
35,41008103.0,41008122.0,1.000000,8,0,4,4-1
70,41008124.0,41008126.0,0.976744,24,4,6,6-2
116,41008105.0,41008117.0,0.958333,3,6,9,19
141,41008107.0,41008106.0,0.954545,0,0,2-4,2-3
177,41008157.0,41008103.0,0.944444,0,8,27,4
212,41008157.0,41008145.0,0.944444,0,1,27,20
248,41008157.0,41008122.0,0.944444,0,0,27,4-1
283,41008122.0,41008145.0,0.942857,0,1,4-1,20
319,41008103.0,41008145.0,0.942857,8,1,4,20


In [41]:
TripChain[TripChain['버스노선ID5']==41008155.0]['버스노선ID5'].count()

0

In [69]:
access_token = 'pk.eyJ1IjoidG9wc29sMTciLCJhIjoiY2szOGQ3MWhoMDdtdDNqb3ZlNHBla2w4bSJ9.lgVDxlf8ndzEDe496xKi0Q'
view_options = {
    'center': [126.908, 37.132],
    'zoom': 12,
    'bearing': 0,
    'pitch': 60,
    'style': 'mapbox://styles/mapbox/dark-v9',
    'access_token': access_token
}

def get_random_color(opacity=1):
    """
        deckgl.Layer 파라미터 중 getColor 에 값을 주는데 쓰입니다.
    """
    rgba = np.random.choice(range(256), size=3).tolist()
    rgba.append(opacity*255)
    return rgba


def draw_path(path_list, name_list=None, colors=[], opacity=1): 
    """
        path_list 로 넘어온 path 들을 한 지도 안에 그립니다.
    """
    # 시각화할 데이터 형태로 가공
    data = []
    for i, path in enumerate(path_list):
        d = {
            'path': path,
            'color': get_random_color() if len(colors) == 0 else colors[i]
        }
        if name_list != None:
            d['name'] = name_list[i]
        data.append(d)
            
    # 지도 설정
    v = view_options.copy()
    v['center'] = np.mean(path_list[0], axis=0)
    
    # 지도그리기
    m = deckgl.Map(**view_options)
    m.add(deckgl.PathLayer(data,
                           getWidth=20,
                           getColor='obj => obj.color',
                           opacity=opacity,
                           pickable=True,
                           tooltip=['name']))
    
    m.show()

In [72]:
df_route_stations

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng,이비노선ID,표준노선ID
998,68834,228000204,24,53,한원IC,233000169,37403.0,화성시,37.152733,127.122883,228000204.0,41006281.0
999,68835,228000204,24,54,장지동,233000168,36808.0,화성시,37.155117,127.117900,228000204.0,41006281.0
1000,68836,228000204,24,55,장지동,233000167,36809.0,화성시,37.155867,127.115017,228000204.0,41006281.0
1001,68837,228000204,24,56,롯데캐슬.제일풍경채,233003130,55526.0,화성시,37.157483,127.109283,228000204.0,41006281.0
1002,68838,228000204,24,57,자이파밀리에,233003131,55527.0,화성시,37.159167,127.106050,228000204.0,41006281.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40237,191750,241491011,38-1,63,신일해피트리후문,233002733,NaN,화성시,37.222650,126.978983,NaN,NaN
40238,191751,241491011,38-1,64,기배동행정복지센터,233002726,NaN,화성시,37.224000,126.983150,NaN,NaN
40239,191752,241491011,38-1,65,기안1통입구,233002959,NaN,화성시,37.225683,126.986250,NaN,NaN
40240,191753,241491011,38-1,66,배양2리,233002717,55455.0,화성시,37.226833,126.992067,NaN,NaN


In [75]:
def get_route_path(route_id):
    idx = df_route_stations['표준노선ID'] == route_id
    return df_route_stations[idx][['lng', 'lat']].values.tolist()

def get_route_name(route_id):
    idx = df_route_stations['표준노선ID'] == route_id
    return df_route_stations[idx]['bus_line_no'].iloc[0]

In [76]:
# 노선 route_1 와 중복되는 노선이 여러개 있으면, 한번에 보여줍니다.
for route_1, gdf in df_topn_pair.groupby('route_1'):
    path_list, name_list = [], []
    path_list.append(get_route_path(route_1))
    name_list.append(get_route_name(route_1))
    
    print(name_list[0], route_1)
    print("-------------------")
    
    # 각 노선이 겹치지 않게, 노선의 위도 경도에 alpha 만큼 더해줍니다.
    alpha = 0.0005
    extra = np.array([alpha, alpha])
    for i, row in gdf.iterrows():
        
        path_list.append((np.array(get_route_path(row['route_2'])) + extra).tolist())
        name_list.append(get_route_name(row['route_2']))
        extra += np.array([alpha, alpha])

        print(name_list[-1], "%.2f"%row['redundancy'])
        
    draw_path(path_list, name_list, opacity=1)

2 41008101.0
-------------------
2-3 0.81


4 41008103.0
-------------------
4-1 1.00
20 0.94


9 41008105.0
-------------------
19 0.96


2-4 41008107.0
-------------------
2-3 0.95


5-2 41008110.0
-------------------
5-4 0.90


...

2-1 41008134.0
-------------------
2-3 0.82
2 0.82


25 41008155.0
-------------------
2-2 0.86
27 0.86
20 0.86
2-1 0.86
4-1 0.86
18 0.86
9 0.86
19 0.86
2-4 0.86
2-3 0.86
4 0.86
2 0.86


27 41008157.0
-------------------
4 0.94
20 0.94
4-1 0.94


1004 41037102.0
-------------------
1000 0.85
330 0.84


6001 41086001.0
-------------------
6003 1.00
